# Fourth small Homework

In this homework I will be getting a file from an online database and use a pretrained network to train it on this database using transfered learning

I used a different database from the suggested one, but this is a google dataset aswell. The result were promising

In [1]:
# Downloading the compressed data that we will be using
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O /tmp/cats_and_dogs_filtered.zip

--2020-11-11 20:49:54--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.15.80, 172.217.12.240, 172.217.9.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.15.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  34.3MB/s    in 1.9s    

2020-11-11 20:49:56 (34.3 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
#import the inception model which is my pretrained model of choice 
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [3]:
# Pretrained model
pre_trained_model = InceptionV3(input_shape = (150,150,3),# Shape of our images
                                include_top = False,# Leave out the last fully connected layer
                                weights = 'imagenet'
                               )

87916544/87910968 [==============================] - 1s 0us/step


In [4]:
# make all layers not trainable (The layers added later will still be trainable which we can see in the learning process)
for layer in pre_trained_model.layers:
    layer.trainable = False

In [5]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras

#Flatten the output layer to 1 dimension
x = keras.layers.Flatten()(pre_trained_model.output)

# Add a fully connected layer with 1024 hidden units an ReLU activation
x = keras.layers.Dense(1024,activation='relu')(x)
# Add a dropout rate of 0.2
x = keras.layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = keras.layers.Dense(1,activation = 'sigmoid')(x)

model = keras.Model(pre_trained_model.input, x)

# compile the model
model.compile(optimizer = RMSprop(lr=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['acc'])

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [7]:
# import relevant packages
import os 
import zipfile

# Unzip the compressed file
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp')
zip_ref.close()


# create the relevant directories
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')

train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_vats_dir = os.path.join(validation_dir, 'cats')

validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [8]:
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

# Create the train generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (150,150))

# Create the validation generator
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        batch_size = 20,
                                                        class_mode = 'binary',
                                                        target_size = (150,150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [9]:
# Creating custom callback
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):

        if(logs.get('acc') > 0.959):
            print("\nReached 99.9% accuracy so cancelling training!") # This should say 95.9%
            self.model.stop_training = True

In [10]:
callbacks = myCallback()

# Running the learning.
history = model.fit_generator(train_generator,
                              validation_data = validation_generator,
                              steps_per_epoch = 100,
                              epochs = 100,
                              validation_steps = 50,
                              verbose = 2,
                              callbacks = [callbacks])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
100/100 - 20s - loss: 1.3410 - acc: 0.8425 - val_loss: 0.2203 - val_acc: 0.9520
Epoch 2/100
100/100 - 19s - loss: 0.4838 - acc: 0.8930 - val_loss: 0.2660 - val_acc: 0.9340
Epoch 3/100
100/100 - 19s - loss: 0.4172 - acc: 0.8975 - val_loss: 0.1994 - val_acc: 0.9490
Epoch 4/100
100/100 - 19s - loss: 0.3724 - acc: 0.9085 - val_loss: 0.1760 - val_acc: 0.9530
Epoch 5/100
100/100 - 19s - loss: 0.3952 - acc: 0.9025 - val_loss: 0.1598 - val_acc: 0.9530
Epoch 6/100
100/100 - 19s - loss: 0.3496 - acc: 0.9085 - val_loss: 0.2077 - val_acc: 0.9490
Epoch 7/100
100/100 - 19s - loss: 0.3152 - acc: 0.9175 - val_loss: 0.2499 - val_acc: 0.9360
Epoch 8/100
100/100 - 19s - loss: 0.3606 - acc: 0.9125 - val_loss: 0.1484 - val_acc: 0.9600
Epoch 9/100
100/100 - 19s - loss: 0.2622 - acc: 0.9255 - val_loss: 0.3081 - val_acc: 0.9330
Epoch 10/100
100/100 - 19s - loss: 0.2680 - acc: 0.9255 - val_loss: 0.1744 - val_acc: 0.9510
Epo

In [ ]:
# Sources:

Sources:

NVIDIA WORKSHOP - there wasa  part on transfered learning which proved useful

https://medium.com/analytics-vidhya/end-to-end-image-classification-project-using-tensorflow-46e78298fa2f

https://medium.com/analytics-vidhya/transfer-learning-using-inception-v3-for-image-classification-86700411251b


